文本分类问题
二分类，也是最基础的分类，顾名思义是将文本归为两种类别，比如将正常邮件邮件划分问题，垃圾邮件或者正常邮件。一段影评，判断是好评还是差评的问题。
多分类，是将文本划分为多个类别，比如将新闻归为政治类，娱乐类，生活类等等。
多标签分类，是给文本贴上多个不同的标签，比如一部小说可以同时被划分为多个主题，可能既是修仙小说，又是玄幻小说



中文邮件分类
1. 导入数据，并进行分词和剔除停用词。
2. 划分训练集和测试集。
3. 将文本数据转化为数字特征数据。
4. 构建分类器。
5. 训练分类器。
6. 测试分类器。


In [1]:
path = 'ham_data.txt'#正常邮件
path2 = 'spam_data.txt'#垃圾邮件
h = open(path,'r',encoding='utf-8')
h_data = h.readlines()
s = open(path2, encoding='utf-8')
s_data = s.readlines()



In [2]:
import sklearn
print(sklearn.__version__)

1.2.2


In [3]:
h

<_io.TextIOWrapper name='ham_data.txt' mode='r' encoding='utf-8'>

In [4]:
h_data[0:3]

['讲的是孔子后人的故事。一个老领导回到家乡，跟儿子感情不和，跟贪财的孙子孔为本和睦。 老领导的弟弟魏宗万是赶马车的。 有个洋妞大概是考察民俗的，在他们家过年。 孔为本总想出国，被爷爷教育了。 最后，一家人基本和解。 顺便问另一类电影，北京青年电影制片厂的。中越战背景。一军人被介绍了一个对象，去相亲。女方是军队医院的护士，犹豫不决，总是在回忆战场上负伤的男友，好像还没死。最后 男方表示理解，归队了。\n',
 '不至于吧，离开这个破公司就没有课题可以做了？ 谢谢大家的关心，她昨天晚上睡的很好。MM她自己已经想好了。见机行事吧，拿到相关的能出来做论文的材料，就马上辞职。 唉！看看吧，说不定还要各为XDJM帮出出找工作的主意呢。MM学通信的，哈尔滨工程大学的研究生，不想在哈碌碌无为的做设计，因此才出来的。先谢谢了啊。！！！ 本人语文不好，没加标点。辛苦那些看不懂的XDJM么了。\n',
 '生一个玩玩，不好玩了就送人 第一，你要知道，你们恋爱前，你爹妈对她是毫无意义的。没道理你爹妈就要求她生孩子，她就得听话。换句话说，你岳父母要未来孩子跟妈姓，你做的到吗？夫妻是平等的。如果你没办法答应岳父母，她干吗答应你爹妈呢？ 第二，有了孩子你养不养的起？不是说想生就生，图你爹妈一个高兴，如果没有房子，没有充足的财力，生孩子只会带给你们更多的困难，生小孩容易，养小孩难啊。\n']

In [5]:
import numpy as np
h_labels = np.ones(len(h_data)).tolist()

In [6]:
s_labels = np.zeros(len(s_data)).tolist()

In [7]:
datas = h_data + s_data
labels = h_labels + s_labels

In [8]:

from sklearn.model_selection import train_test_split

In [9]:
train_d,test_d,train_y,test_y = train_test_split(datas,labels,test_size=0.25,random_state=5)

In [10]:
import jieba


def tokenize_words(corpus):
    tokenized_words = jieba.cut(corpus)
    tokenized_words = [token.strip() for token in tokenized_words]
    return tokenized_words


In [11]:
def remove_stopwords(corpus):  # 函数输入为样本集
    sw = open('stop_word.txt', encoding='utf-8')  # stopwords 停词表
    sw_list = [l.strip() for l in sw]  # 去掉文本中的回车符，然后存放到 sw_list 当中
    # 调用前面定义好的分词函数返回到 tokenized_data 当中
    tokenized_data = tokenize_words(corpus)
    # 过滤停用词，对每个在 tokenized_data 中的词 data 进行判断
    # 如果 data 不在 sw_list 则添加到 filtered_data 当中
    filtered_data = [data for data in tokenized_data if data not in sw_list]
    # 用''将 filtered_data 串起来赋值给 filtered_datas
    filtered_datas = ' '.join(filtered_data)
    return filtered_datas  # 返回去停用词之后的 datas


In [12]:
from tqdm.notebook import tqdm


def preprocessing_datas(datas):
    preprocessed_datas = []
    # 对 datas 当中的每一个 data 进行去停用词操作
    # 并添加到上面刚刚建立的 preprocessed_datas 当中
    for data in tqdm(datas):
        data = remove_stopwords(data)
        preprocessed_datas.append(data)

    return preprocessed_datas  # 返回去停用词之后的新的样本集


In [13]:
pred_train_d = preprocessing_datas(train_d)



  0%|          | 0/7500 [00:00<?, ?it/s]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\luoso\AppData\Local\Temp\jieba.cache
Loading model cost 1.065 seconds.
Prefix dict has been built successfully.


In [15]:
pred_test_d = preprocessing_datas(test_d)


  0%|          | 0/2501 [00:00<?, ?it/s]

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    min_df=1, norm='l2', smooth_idf=True, use_idf=True, ngram_range=(1, 1))


In [17]:
tfidf_train_features = vectorizer.fit_transform(pred_train_d)
tfidf_train_features


<7500x28336 sparse matrix of type '<class 'numpy.float64'>'
	with 199263 stored elements in Compressed Sparse Row format>

In [18]:
tfidf_train_features.toarray()[0]
#查看训练特征集

array([0., 0., 0., ..., 0., 0., 0.])

In [20]:
tfidf_test_features = vectorizer.transform(pred_test_d)



In [19]:
from sklearn.linear_model import SGDClassifier

svm = SGDClassifier(loss='hinge')
svm.fit(tfidf_train_features, train_y)




SGDClassifier()

In [21]:
predictions = svm.predict(tfidf_test_features)

In [23]:
from sklearn import metrics

accuracy_score = np.round(metrics.accuracy_score(test_y, predictions), 2)
accuracy_score


0.99

In [24]:
print('邮件类型：', test_y[20])
print('预测邮件类型：', predictions[20])
print('文本：', test_d[20])


邮件类型： 0.0
预测邮件类型： 0.0
文本： 您好！ 我公司有多余的发票可以向外代开！（国税、地税、运输、广告、海关缴款书）。 如果贵公司（厂）有需要请来电洽谈、咨询！ 联系电话：(0)13510614952 吴小姐 谢谢！ 顺祝商祺!

